In [50]:
!pip install prophet

In [51]:
import pandas as pd
from google.colab import drive
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import os

In [79]:
# Đường dẫn tới file trên Google Drive
train_file_path = '/content/drive/MyDrive/LuanVan/data/ComVN30_train.csv'
test_file_path = '/content/drive/MyDrive/LuanVan/data/ComVN30_test.csv'
# Đọc dữ liệu
train_df = pd.read_csv(train_file_path)
test_df = pd.read_csv(test_file_path)

In [80]:
class StockModelEvaluation:
    def __init__(self):
        self.results = []

    def fit_and_predict(self, model, train_df, test_df, ticker):
        # Huấn luyện mô hình với dữ liệu train
        model.fit(train_df)

        # Dự đoán với tập test
        future = test_df[['ds']]
        forecast = model.predict(future)

        # Điều chỉnh giá trị dự đoán
        last_train_value = train_df['y'].iloc[-1]
        first_forecast_value = forecast['yhat'].iloc[0]
        adjustment = last_train_value - first_forecast_value
        forecast['yhat_adjusted'] = forecast['yhat'] + adjustment
        forecast['yhat_upper_adjusted'] = forecast['yhat_upper'] + adjustment
        forecast['yhat_lower_adjusted'] = forecast['yhat_lower'] + adjustment

        # Tính toán MAE và RMSE
        y_true = test_df['y'].values
        y_pred = forecast['yhat_adjusted'].values

        mae = round(mean_absolute_error(y_true, y_pred), 2)
        rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)

        print(f'Ticker: {ticker}, MAE: {mae}, RMSE: {rmse}')

        # Lưu kết quả
        self.results.append({
            'ticker': ticker,
            'mae': mae,
            'rmse': rmse,
            'seasonality_mode': model.seasonality_mode
        })

        # Trả về dự báo đã điều chỉnh và thang đo đánh giá
        return forecast

    def plot_forecast(self, train_df, forecast, test_df, ticker, seasonality_mode):
        # Xác định xu hướng dự đoán
        start_value = forecast['yhat_adjusted'].iloc[0]
        end_value = forecast['yhat_adjusted'].iloc[-1]
        trend = 'up' if end_value > start_value else 'down' if end_value < start_value else 'stable'

        # Tên file ảnh
        title = f'{seasonality_mode.capitalize()} Prophet_{ticker}: Trend {trend}'
        filename = title.replace(" ", "").replace(":", "_").replace(",", "") + '.png'
        file_path = os.path.join('/content/drive/MyDrive/LuanVan/results/FB_Prophet/', filename)

        # Vẽ biểu đồ
        plt.figure(figsize=(10, 6))
        plt.plot(train_df['ds'], train_df['y'], label='Train', color='blue')
        plt.plot(forecast['ds'], forecast['yhat_adjusted'], label='Prediction', color='orange')
        plt.fill_between(forecast['ds'], forecast['yhat_lower_adjusted'], forecast['yhat_upper_adjusted'], color='orange', alpha=0.2)
        plt.plot(test_df['ds'], test_df['y'], label='Test', color='green')
        plt.title(title)
        plt.xlabel('Date')
        plt.ylabel('Close Price')
        plt.legend()
        plt.show()

        # Lưu biểu đồ vào file
        plt.savefig(file_path)
        plt.close()

    def get_results(self, seasonality_mode):
        # Lọc kết quả theo seasonality_mode
        filtered_results = [result for result in self.results if result['seasonality_mode'] == seasonality_mode]
        results_df = pd.DataFrame(filtered_results)

        # Tính toán trung bình MAE và RMSE
        avg_mae = round(results_df['mae'].mean(), 2)
        avg_rmse = round(results_df['rmse'].mean(), 2)

        print(f'Average MAE ({seasonality_mode.capitalize()}): {avg_mae}, Average RMSE ({seasonality_mode.capitalize()}): {avg_rmse}')

        # Lưu kết quả vào tệp CSV
        csv_path = f'/content/drive/MyDrive/LuanVan/results/FB_Prophet/Prophet_results_{seasonality_mode}.csv'
        results_df.to_csv(csv_path, index=False)

        return results_df, avg_mae, avg_rmse


In [81]:
# Chuyển đổi cột 'time' sang định dạng datetime
train_df['time'] = pd.to_datetime(train_df['time'])
test_df['time'] = pd.to_datetime(test_df['time'])

# Tạo đối tượng StockModelEvaluation
evaluator_additive = StockModelEvaluation()
evaluator_multiplicative = StockModelEvaluation()

# Lấy danh sách các mã chứng khoán
tickers = train_df['ticker'].unique()

## Additive

In [82]:
# Vòng lặp qua từng mã chứng khoán với additive seasonality
for ticker in tickers:
    print(f"Processing ticker (Additive): {ticker}")

    # Chuẩn bị dữ liệu cho từng mã chứng khoán
    train_df_ticker = train_df[train_df['ticker'] == ticker]
    test_df_ticker = test_df[test_df['ticker'] == ticker]

    # Đổi tên cột cho phù hợp với yêu cầu của Prophet
    train_df_ticker = train_df_ticker.rename(columns={'time': 'ds', 'close': 'y'})
    test_df_ticker = test_df_ticker.rename(columns={'time': 'ds', 'close': 'y'})

    # Chỉ lấy các cột cần thiết
    train_df_ticker = train_df_ticker[['ds', 'y']]
    test_df_ticker = test_df_ticker[['ds', 'y']]

    # Tạo mô hình Prophet với các thành phần mùa vụ hàng tuần và hàng năm
    model_additive = Prophet(yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=False, seasonality_mode='additive')

    # Huấn luyện mô hình và dự báo
    forecast_additive = evaluator_additive.fit_and_predict(model_additive, train_df_ticker, test_df_ticker, ticker)

    # Vẽ biểu đồ dự báo
    evaluator_additive.plot_forecast(train_df_ticker, forecast_additive, test_df_ticker, ticker, 'Additive')


Output hidden; open in https://colab.research.google.com to view.

## Multiplicative

In [83]:

# Vòng lặp qua từng mã chứng khoán với multiplicative seasonality
for ticker in tickers:
    print(f"Processing ticker (Multiplicative): {ticker}")

    # Chuẩn bị dữ liệu cho từng mã chứng khoán
    train_df_ticker = train_df[train_df['ticker'] == ticker]
    test_df_ticker = test_df[test_df['ticker'] == ticker]

    # Đổi tên cột cho phù hợp với yêu cầu của Prophet
    train_df_ticker = train_df_ticker.rename(columns={'time': 'ds', 'close': 'y'})
    test_df_ticker = test_df_ticker.rename(columns={'time': 'ds', 'close': 'y'})

    # Chỉ lấy các cột cần thiết
    train_df_ticker = train_df_ticker[['ds', 'y']]
    test_df_ticker = test_df_ticker[['ds', 'y']]

    # Tạo mô hình Prophet với các thành phần mùa vụ hàng tuần và hàng năm
    model_multiplicative = Prophet(yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=False, seasonality_mode='multiplicative')

    # Huấn luyện mô hình và dự báo
    forecast_multiplicative = evaluator_multiplicative.fit_and_predict(model_multiplicative, train_df_ticker, test_df_ticker, ticker)

    # Vẽ biểu đồ dự báo
    evaluator_multiplicative.plot_forecast(train_df_ticker, forecast_multiplicative, test_df_ticker, ticker, 'Multiplicative')


Output hidden; open in https://colab.research.google.com to view.

In [84]:
# Lấy và hiển thị kết quả
results_df_additive, avg_mae_additive, avg_rmse_additive = evaluator_additive.get_results('additive')
results_df_multiplicative, avg_mae_multiplicative, avg_rmse_multiplicative = evaluator_multiplicative.get_results('multiplicative')

print("Additive Results:")
print(results_df_additive)
print(f'Average MAE (Additive): {avg_mae_additive}, Average RMSE (Additive): {avg_rmse_additive}')

print("Multiplicative Results:")
print(results_df_multiplicative)
print(f'Average MAE (Multiplicative): {avg_mae_multiplicative}, Average RMSE (Multiplicative): {avg_rmse_multiplicative}')



Average MAE (Additive): 3358.7, Average RMSE (Additive): 4003.01
Average MAE (Multiplicative): 3887.96, Average RMSE (Multiplicative): 4605.6
Additive Results:
   ticker       mae      rmse seasonality_mode
0     SSI   1531.85   1783.62         additive
1     BCM   3813.82   4323.65         additive
2     VHM   1301.49   1651.37         additive
3     VIC   3834.42   4917.44         additive
4     VRE   1686.07   1976.67         additive
5     BVH   3427.81   3956.37         additive
6     POW    189.98    244.24         additive
7     GAS   4600.38   6024.80         additive
8     ACB   2142.07   2259.26         additive
9     BID   4879.13   5675.99         additive
10    CTG   5226.09   5538.13         additive
11    HDB   1587.30   1850.21         additive
12    MBB   3432.73   3652.41         additive
13    SSB   1081.33   1199.97         additive
14    SHB   1062.76   1257.87         additive
15    STB    938.67   1505.47         additive
16    TCB   6794.14   8229.12         add